## NLG Task
### Tatiana Shavrina

To run this notebook, please change the data directory in main()

In [ ]:
import re
import os
from nltk.tokenize import word_tokenize
import sqlite3
from bs4 import BeautifulSoup as bs

In this notebook I would like to illustrate, how a QA-system can work, parsing triplets and putting them into database, as this is a fast way to get specified information

In [ ]:
def search(lst, cur, con):
    found_lst = []
    status = ''
    #try:
    for w in lst:
        try:
            cur.execute("SELECT * FROM countries, cities where countries.countryname = cities.country and countries.countryname="+w)
            status = 'country'
            found_lst += cur.fetchall()
            
            con.commit()
        except:
            pass
            
        try:
        
            cur.execute("SELECT * FROM countries, cities where countries.countryname = cities.country and countries.language="+w)
            status = 'language'
            found_lst += cur.fetchall()
           
            con.commit()
        except:
            pass
            
        try:
            cur.execute("SELECT * FROM countries, cities where countries.countryname = cities.country and cities.cityname="+w)
            status = 'city'
            found_lst += cur.fetchall()
            
            con.commit()
        except:
            pass
            
        try:
            cur.execute("SELECT * FROM countries, cities where countries.countryname = cities.country and cities.airport="+w)
            status = 'airport'
            found_lst += cur.fetchall()
            
            con.commit()
        except:
            pass
            
        if len(found_lst)>0:
            break
    """except:
        found_lst = []
        status = ''"""
    return found_lst, status

In [ ]:
def question_type(words):
    loc = ['where', 'what', 'is', 'situated', 'located', 'location', 'which', 'country', 'city']
    lang = ['where', 'what', 'which', 'is', 'spoken', 'speak', 'language']
    result1 = list(set(words) & set(loc))
    result2 = list(set(words) & set(lang))
    if len(result1)>len(result2):
        return 'loc'
    else:
        return 'lang'

In [ ]:
def chatbot(question, cur, con):
    words = word_tokenize(question)
    data, status = search(words, cur, con)
    qtype = question_type(words) #location or language
    if data!=[]:
        if status=='country':
            if qtype == 'loc':
                return "Embarassing question..."
            else:
                return data[1]+" is spoken in "+data[0]
        elif status=='language':
            if qtype == 'loc':
                return data[1]+" is spoken in "+data[0]
            else:
                return "Embarassing question..."
        elif status=='city':
            if qtype == 'loc':
                return data[2] +" is situated in "+data[0]
            else:
                return data[1]+" is spoken in "+data[2]
        elif status=='airport':
            if qtype == 'loc':
                return data[-1] + " is located in "+data[2] +", "+data[0]
            else:
                return "In airports they often speak English :)"
    else:
        return "I dont' know"
    
    

In [ ]:
def getdata(datadir):
    
    traindir = datadir  + '/train/3triples/3triples_Airport_train_challenge.xml'
    testdir = datadir + '/dev/3triples/3triples_Airport_dev_challenge.xml'
    #filedir = join(traindir, file)
    train = bs(open(traindir).read(), "lxml")
    test = bs(open(testdir).read(), "lxml")
    
    train1 = train.benchmark.entries.findAll("entry")
    test1 = test.benchmark.entries.findAll("entry")
    
    return train1, test1

In [ ]:
def getfeatures(xml):
    con = sqlite3.connect('NLG.db')
    cur = con.cursor()
    cur.execute("CREATE TABLE CITIES ( CITYNAME VARCHAR  NOT NULL, COUNTRY VARCHAR, AIRPORT VARCHAR)")
    cur.execute("CREATE TABLE COUNTRIES (COUNTRYNAME VARCHAR NOT NULL, LANGUAGE VARCHAR)")
    
    lst = xml
    
    for i in range(len(lst)):
        triplets = [elem.split('</mtriple>')[0]  for elem in str(lst[i]).split('<mtriple>')[1:] ]
        cityname,airport,country,language = '', '', '', ''
        for j in triplets:
            
            l = j.split(' | ')
            
            if l[1]=="location" :
                cityname = l[2].lower()# first - cityname
                airport = l[2].lower()
                
                
            elif l[1]=="country":
                
                country = l[2].lower()
            elif l[1]=='language':
                
                language = l[2].lower().replace('_',' ')
            
        cur.execute("INSERT INTO cities values (?, ?, ?)", (cityname, country, language))
        cur.execute("INSERT INTO countries values (?, ?)", (country, language))
        con.commit()
                
    
    return con, cur

In [ ]:
def __main__():
    tf = True
    datadir = r'/home/mi_air/Documents/HSE/corpus_linguistics/NLG/challenge_data_train_dev'
    train, test = getdata(datadir)
    #global train
    #global test
    con.commit()
    while tf:
        question = str(input('Place your question here: ')).lower()
        if question !="":
            if question=="q":
                tf = False
                print('Thank you! Buy!')
            else:
                answer = chatbot(question, cur, con)
                print(answer + '\n')
                
        else:
            question = str(input('Place your question here: (to quit, type Q)'))

In [ ]:
if __name__==__main__():
    __main__()